In [1]:
import os
import pandas as pd
from datetime import datetime, timedelta, date, time
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
# Parameters
nb_name = "2a_build_omie_matrix_out"
config_path = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/configs/config.yaml"
ssaa_path_df = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/processed/elec/ssaa_c2.pkl"
perd_path = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/processed/elec/perdidas.pkl"
spot_path = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/processed/elec/spot_elec.parquet"
spot_formated_path = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/processed/elec/spot_h_periodos.pkl"
futures_path = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/processed/elec/futuros_elec.parquet"
path_omie_matrix = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/verification_project/elec/omie_matrix_verif.pkl"


In [ ]:
from config.config_loader import Config

config = Config(config_file=config_path)

In [ ]:
from utils.logging_setup import get_logger
import time as tm

instant = tm.strftime("%Y-%m-%d_%H:%M:%S", tm.localtime())
logger = get_logger(name=f"{nb_name}")
logger.info(f"Building omie matrix on {instant}")

In [4]:
df_ssaa = pd.read_pickle(ssaa_path_df)
df_ssaa = df_ssaa.copy()
df_ssaa.loc[:, 'dup'] = df_ssaa.groupby('datetime').cumcount()
date_max = df_ssaa['datetime'].max()

df_spot = pd.read_pickle(spot_formated_path)
# df_spot_fil = df_spot[(df_spot['year'] == year) & (df_spot['month'] == month)]
df_spot_fil = df_spot[df_spot['datetime'] <= date_max]
df_spot_fil.reset_index(drop=True, inplace=True)
df_spot_fil.reset_index(drop=False, inplace=True)
df_spot_fil = df_spot_fil.copy()
df_spot_fil.loc[:, 'dup'] = df_spot_fil.groupby('datetime').cumcount()

df_merge = pd.merge(df_spot_fil, df_ssaa[['datetime', 'dup', 'ssaa']], on=['datetime', 'dup'], how='left')
df_merge.drop('index', axis=1, inplace=True)
df_merge.groupby('periodo')['ssaa'].mean()
df_merge = df_merge[['datetime', 'periodo', 'value', 'ssaa', 'dup']]
logger.info('Merged ssaa df to spot df')

,datetime,periodo,value,ssaa,dup
0,2014-01-01 00:00:00,6,20.02,NaN,0
1,2014-01-01 01:00:00,6,10.34,NaN,0
2,2014-01-01 02:00:00,6,5.35,NaN,0
3,2014-01-01 03:00:00,6,5.00,NaN,0
4,2014-01-01 04:00:00,6,0.50,NaN,0
...,...,...,...,...,...
100045,2025-05-31 19:00:00,6,31.78,17.172741,0
100046,2025-05-31 20:00:00,6,62.00,15.426091,0
100047,2025-05-31 21:00:00,6,64.31,16.026816,0
100048,2025-05-31 22:00:00,6,74.44,14.124158,0


In [8]:
df_perd = pd.read_pickle(perd_path)
df_perd = df_perd.copy()
df_perd.loc[:, 'dup'] = df_perd.groupby('datetime').cumcount()
df_perd.tail()

,datetime,20_td,30_td,61_td,62_td,63_td,64_td,dup
21162,2025-05-31 19:00:00,0.180,0.180,0.077,0.054,0.044,0.017,0
21163,2025-05-31 20:00:00,0.177,0.177,0.076,0.053,0.043,0.017,0
21164,2025-05-31 21:00:00,0.179,0.179,0.077,0.054,0.044,0.017,0
21165,2025-05-31 22:00:00,0.182,0.182,0.078,0.055,0.044,0.017,0
21166,2025-05-31 23:00:00,0.185,0.185,0.079,0.055,0.045,0.017,0


In [10]:
df_matrix = pd.merge(df_perd, df_merge, on=['datetime', 'dup'])
df_matrix.drop('dup', axis=1, inplace=True)
df_matrix = df_matrix[['datetime', 'periodo', 'value', 'ssaa', '20_td', '30_td', '61_td', '62_td', '63_td', '64_td']]
logger.info('Merged perd df to spot and ssaa previously merged df')

In [ ]:
df_matrix.to_pickle(path_omie_matrix)
logger.info(f"Omie matrix saved to {path_omie_matrix}")
logger.info("Omie matrix creation completed successfully.")